# Create Response Generator Dataset
This notebook creates the dataset for re-training the response generator model.

## Pre-requisites
A CSV file must be created that contains fields including input, search results, and output.

In [ ]:
from citation_fetcher import Citation_Fetcher as cf
from query_parser import QueryParser as QP
import const
import numpy as np
import pandas as pd

In [ ]:
RESPONSE_GEN_DS_PATH: str =  const.DATASETS_FOLDER + "response_gen_ground_truth.csv"
SQUAD_DS_PATH: str = const.DATASETS_FOLDER + "squad_ds_keyword_ground_truth.csv"
ULTRACHAT_LITE_PATH: str = const.DATASETS_FOLDER + "ultrachat_50k.csv"
REDDIT_DS_PATH: str = const.DATASETS_FOLDER + "reddit_ds.csv"

In [ ]:
squad_ds: pd.DataFrame = pd.read_csv(SQUAD_DS_PATH)
squad_len = len(squad_ds)

In [ ]:
ultrachat_ds: pd.DataFrame = pd.read_csv(ULTRACHAT_LITE_PATH)
ultrachat_len = len(squad_ds)

In [ ]:
reddit_ds: pd.DataFrame = pd.read_csv(REDDIT_DS_PATH)

In [ ]:
rg_ds = squad_ds['question']

In [ ]:
rg_ds = pd.DataFrame(data={'input': rg_ds, 'query': None, 'search_result': None, 'output': None})

In [ ]:
rg_ds.head()

In [ ]:
ultrachat_ds.head()

In [ ]:
ultrachat_ds = ultrachat_ds.rename(columns={'question': 'input'})

In [ ]:
rg_ds  = pd.concat([rg_ds, ultrachat_ds['input']], ignore_index=True)

In [ ]:
i = 0
qp = QP(const.MODELS_FOLDER + "aletheianomous_ai-keyword_extractor-v0.3.1/")
for j in range(len(rg_ds)):
    print("Populating Search Result ", (int((i/len(rg_ds))*100)),"% (Sample ", (i+1), "/", len(rg_ds) , ")", 
                end="                   \r")
    user_input = rg_ds.loc[j, "input"]
    query = qp.gen_query(user_input)
    json_out, result = cf.search_online(query)
    rg_ds[j, "search_result"] = result
    i+=1

In [ ]:
reddit_ds.head()

In [ ]:
reddit_ds = reddit_ds.rename(columns={"0": "input"})

In [ ]:
rg_ds = pd.concat([rg_ds, reddit_ds['input']], ignore_index=True)

In [ ]:
rg_ds.to_csv(RESPONSE_GEN_DS_PATH)